In [5]:
# 라이브러리 로드
import pickle as pkl
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook



def interpolated_data_load(link):
    # 데이터 불러오기
    data = '{}crw/non_err_data_convert5.pkl'.format(link)
    with open(data, 'rb')as f:
        data = pkl.load(f)
    data.set_index('A0', inplace = True)
    data.sort_index(inplace=True)

    # 2010~2019만 사용 - 3개 이상 드랍, 2개 이하부터는 대충 채워줌ㅋ
    price_data = data.iloc[:,:30].astype('float').T # 앞뒤 데이터의 기울기로 데이터 채우기
    price_data = price_data/1000
    for i in tqdm_notebook(price_data.columns) :
        price_data[i] = price_data[i].interpolate()

    price_data = price_data[20:30] # 마지막 10년도 데이터만 사용 (2010~2019)
    #print("Drop 하는 양 :", price_data[price_data.columns[(price_data.isnull().sum()>=3).tolist()]].shape[1])
    price_data.drop(price_data.columns[(price_data.isnull().sum()>=3).tolist()], axis=1, inplace=True) # 3개 이상 비면 드랍
    if price_data[9:].isnull().sum().sum() != 0: # 맨 뒤에는 null이 없는가?
        print('2019에 null이 있다!')

    price_data.iloc[1,:].fillna((price_data.iloc[2,:]*2) - price_data.iloc[3,:], inplace=True)
    price_data.iloc[0,:].fillna((price_data.iloc[1,:]*2) - price_data.iloc[2,:], inplace=True)
    if price_data.isnull().sum().sum() != 0 : # 전체 데이터에서 아직도 null이 있는가?
        print('데이터에 아직도 null이 있다!')
    
    data.drop(data.columns[:20], axis=1, inplace=True)
    data.iloc[:,:10] = price_data.astype('int').T
    print(data.index[(data.iloc[:,:10].T.isnull().sum()>=1).tolist()].shape, "개 만큼 drop함")
    data.drop(data.index[(data.iloc[:,:10].T.isnull().sum()>=1).tolist()], inplace=True) # price에서 drop한게 반영이 안돼있기때문
    for i in data.columns[:10]:
        data.loc[data[data.loc[:,i] < 0].index,i] = 0 # 가격이 음수일 경우 0으로 치환함(사실 급격히 감소하는 친구들 일 것임)

    dmax = data[data.columns[:10]].max().max()
    dmin = data[data.columns[:10]].min().min()
    print("필지 가격의 MIN - MAX는 {} - {}입니다.".format(dmin,dmax))
    dropcal = []
    for i in data.columns[14:]:
        if len(data[i].unique())==1:
            dropcal.append(i)
    data.drop(dropcal, axis=1, inplace = True)
    collist = data.columns[14:]
    for i in data.columns[:10]: # Min-Max정규화
        data[i] = (data[i]-dmin)/(dmax-dmin) 
        for j in collist:
            data[j].replace(0, np.nan, inplace=True)
            data['{}_{}'.format(i,j)] = data[i] * data[j]
            data['{}_{}'.format(i,j)] = data['{}_{}'.format(i,j)] - data['{}_{}'.format(i,j)].mean(skipna=True)
            data[j].replace(np.nan, 0, inplace=True)
            data['{}_{}'.format(i,j)].replace(np.nan, 0, inplace=True)
    data.drop(collist, axis=1, inplace = True)
    
    for i in data.columns[10:14]: # 이것들은 null값을 0으로 채워줌
        if i == 'b_at_size' :
            data.b_at_size = data.b_at_size.replace('',0) # err의 주범
        data[i] = data[i].astype('float').fillna(float(0))
        data[i] = (data[i]-data[i].min())/(data[i].max()-data[i].min()) # Min-Max정규화
    
    y = data[data.columns[:10]].copy(deep=True)
    data.drop(data.columns[:10], axis=1, inplace=True)
    print('data Load Done!')
    return data , y



def xy_load(link):
    # 데이터 불러오기
    xy = '{}crw/addr_xy.pkl'.format(link)
    with open(xy, 'rb')as f:
        xy = pkl.load(f)
    xy.drop(33673, inplace = True) # 임의 중복 제거
    xy.set_index('A0', inplace = True)
    xy.sort_index(inplace=True)
    xy = xy.loc[data.index, :]

    xy['xcoord'] = xy['xcoord'].astype('float')
    xy['ycoord'] = xy['ycoord'].astype('float')
    xy['xcoord'] = xy['xcoord']-202000
    xy['ycoord'] = xy['ycoord']-550000
    print('xy Load Done!')
    return xy



link = 'D:/real/'
data, y = interpolated_data_load(link)
xy = xy_load(link)

data_y = '{}adj/y.pkl'.format(link)
with open(data_y, 'wb') as f : 
    pkl.dump(y, f)


(4590,) 개 만큼 drop함
필지 가격의 MIN - MAX는 0.0 - 30300.0입니다.
data Load Done!
xy Load Done!


In [9]:
# 필요없는 거리열 지우기 - 두번 돌릴 필요 없음

def make_xy_rg_table(data, xy, rg):
    ddmglist = xy[xy['ddm']==1][['xcoord', 'ycoord']]
    nonddmglist = xy[xy['ddm']==0][['xcoord', 'ycoord']]
    nonddmglist.sort_index(inplace=True)
    non_id = nonddmglist.index.tolist()

    drop_id = []
    non_dl = xy[['xcoord', 'ycoord']]
    for j in tqdm_notebook(non_id) :
        if sum(round((abs(ddmglist['xcoord']-nonddmglist.loc[str(j),'xcoord'])**2 + abs(ddmglist['ycoord']-nonddmglist.loc[str(j),'ycoord'])**2)**0.5,0).astype('int') < rg) == 0 :
            drop_id.append(j)
    xyt = xy
    xyt = xyt.T
    xyt = xyt[xyt.columns.difference(drop_id)].T
    
    data = data.T
    data = data[data.columns.difference(drop_id)].T
    
    dropcal = []
    for i in data.columns:
        if len(data[i].unique())==1:
            dropcal.append(i)
    print(len(dropcal)," 개 컬럼은 값이 1가지로 나타나기 때문에 드랍하겠음!")
    data.drop(dropcal, axis=1, inplace = True)
    
    return data, xyt



rg = 1000 # 미만 반경 설정 (1000 or 500)
new_rg_data, new_rg_xy = make_xy_rg_table(data, xy, rg) # 이 열 주석 풀면 실행

xy_range = '{}adj/xy_{}.pkl'.format(link,rg)
with open(xy_range, 'wb') as f : 
    pkl.dump(new_rg_xy, f)

data_range = '{}adj/data_{}.pkl'.format(link,rg)
with open(data_range, 'wb') as f : 
    pkl.dump(new_rg_data, f)

    

rg = 500 # 미만 반경 설정 (1000 or 500)
new_rg_data, new_rg_xy = make_xy_rg_table(data, xy, rg) # 이 열 주석 풀면 실행

xy_range = '{}adj/xy_{}.pkl'.format(link,rg)
with open(xy_range, 'wb') as f : 
    pkl.dump(new_rg_xy, f)

data_range = '{}adj/data_{}.pkl'.format(link,rg)
with open(data_range, 'wb') as f : 
    pkl.dump(new_rg_data, f)


['2010_la_공공주택지구', '2010_la_공장설립승인지역', '2010_la_공장설립제한지역', '2010_la_도시개발구역', '2010_la_문화지구', '2010_la_비행안전제3구역', '2010_la_사고지', '2010_la_역사문화환경보존지역', '2010_la_온천공보호구역', '2010_la_정비예정구역', '2010_la_토지구획정리사업지구', '2010_la_하천미분류', '2010_lu_도시자연공원구역', '2010_lu_박물관', '2010_lu_방송통신시설', '2010_lu_산업개발진흥지구', '2010_lu_연결녹지', '2010_lu_용도구역', '2010_lu_유통업무설비', '2010_lu_일반미관지구', '2010_lu_전용주거지역', '2010_lu_제1종전용주거지역', '2010_lu_준공업지역', '2010_lu_청소년수련시설', '2010_lu_특수도로', '2010_lu_특화경관지구', '2010_lu_폐기물처리시설', '2011_la_공공주택지구', '2011_la_공장설립승인지역', '2011_la_공장설립제한지역', '2011_la_도시개발구역', '2011_la_문화지구', '2011_la_비행안전제3구역', '2011_la_사고지', '2011_la_역사문화환경보존지역', '2011_la_온천공보호구역', '2011_la_정비예정구역', '2011_la_토지구획정리사업지구', '2011_lu_도시자연공원구역', '2011_lu_박물관', '2011_lu_방송통신시설', '2011_lu_산업개발진흥지구', '2011_lu_연결녹지', '2011_lu_용도구역', '2011_lu_유통업무설비', '2011_lu_일반미관지구', '2011_lu_전용주거지역', '2011_lu_제1종전용주거지역', '2011_lu_준공업지역', '2011_lu_청소년수련시설', '2011_lu_특수도로', '2011_lu_특화경관지구', '2011_lu_폐기물처리시설', '2012_la_공공주택지구', '2012_la_


['2010_la_개발제한구역', '2010_la_건축허가·착공제한지역', '2010_la_공공주택지구', '2010_la_공장설립승인지역', '2010_la_공장설립제한지역', '2010_la_도시개발구역', '2010_la_도시용도지역지구용도지역지구', '2010_la_문화지구', '2010_la_비행안전제3구역', '2010_la_사고지', '2010_la_시도지정문화재구역', '2010_la_역사도심', '2010_la_역사문화환경보존지역', '2010_la_온천공보호구역', '2010_la_정비예정구역', '2010_la_토지구획정리사업지구', '2010_la_하천미분류', '2010_lu_개발행위허가제한지역', '2010_lu_대학', '2010_lu_도로시설', '2010_lu_도시자연공원', '2010_lu_도시자연공원구역', '2010_lu_박물관', '2010_lu_방송통신시설', '2010_lu_보행자전용도로', '2010_lu_산업개발진흥지구', '2010_lu_수질오염방지시설', '2010_lu_연결녹지', '2010_lu_용도구역', '2010_lu_운동장', '2010_lu_유통업무설비', '2010_lu_일반도로', '2010_lu_일반미관지구', '2010_lu_전용주거지역', '2010_lu_제1종전용주거지역', '2010_lu_주간선도로', '2010_lu_준공업지역', '2010_lu_청소년수련시설', '2010_lu_체육시설', '2010_lu_특수도로', '2010_lu_특화경관지구', '2010_lu_폐기물처리시설', '2011_la_개발제한구역', '2011_la_건축허가·착공제한지역', '2011_la_공공주택지구', '2011_la_공장설립승인지역', '2011_la_공장설립제한지역', '2011_la_도시개발구역', '2011_la_도시용도지역지구용도지역지구', '2011_la_문화지구', '2011_la_비행안전제3구역', '2011_la_사고지', '2011_la_시도지정문화재구역', '2011_la_역사도

In [18]:
import geopandas as gpd
import pandas as pd
import pickle as pkl
import numpy as np

trp_xy = gpd.read_file('C:/Users/User_T816/Desktop/traffic_data/train_xy_shp.shp', encoding='euc-kr')
trp_xy = pd.DataFrame(np.array(trp_xy), columns = trp_xy.columns)
link = 'C:/Users/User_T816/Desktop/traffic_data/'
with open('{}shp_trp_xy.pkl'.format(link), 'wb') as f : 
    pkl.dump(trp_xy, f)
print('Done!')

Done!


In [12]:
# 라이브러리 로드
import pickle as pkl
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook



def make_adj(xy, rg):
    # 데이터 나누기
    ddmglist = xy[xy['ddm']==1][['xcoord', 'ycoord']]
    ddmglist.sort_index(inplace=True)
    inid = ddmglist.index.tolist()

    in_devide = []
    for i in range(len(inid)):
        if i == 0 :
            cache = 0
        elif i%4000 == 0 :
            in_devide.append(inid[cache:i])
            cache = i
        if i==len(inid)-1:
            in_devide.append(inid[cache:])


    # 실제 연산 부분
    for n,i in enumerate(in_devide) : # 세로로 긴 직사각형
        dl = xy[['xcoord', 'ycoord']] # dl(dividelist)초기화
        for j in tqdm_notebook(i) :
            dl[j] = (((abs(dl['xcoord']-dl.loc[str(j),'xcoord'])**2 + abs(dl['ycoord']-dl.loc[str(j),'ycoord'])**2)**0.5)/rg).apply(lambda x:0 if x>=1 else 1-x)
        dl.drop(['xcoord', 'ycoord'], axis=1, inplace=True)
        check=dl
        with open('D:/real/adj/ddmg_adj_{}_{}.pkl'.format(rg, n), 'wb') as f : 
            pkl.dump(dl, f)
        print('{}/{} Save Done!'.format(n+1, len(in_devide)))
    return 0



# 데이터 불러오기
link = 'D:/real/'
rg = 1000

data_range = '{}adj/data_{}.pkl'.format(link,rg)
with open(data_range, 'rb')as f:
    data = pkl.load(f)
data.sort_index(inplace=True)

xy_range = '{}adj/xy_{}.pkl'.format(link, rg)
with open(xy_range, 'rb')as f:
    xy = pkl.load(f)
xy.sort_index(inplace=True)

make_adj(xy, rg)



# 데이터 불러오기
link = 'D:/real/'
rg = 500

data_range = '{}adj/data_{}.pkl'.format(link,rg)
with open(data_range, 'rb')as f:
    data = pkl.load(f)
data.sort_index(inplace=True)

xy_range = '{}adj/xy_{}.pkl'.format(link, rg)
with open(xy_range, 'rb')as f:
    xy = pkl.load(f)
xy.sort_index(inplace=True)

make_adj(xy, rg)


1/11 Save Done!



2/11 Save Done!



3/11 Save Done!



4/11 Save Done!



5/11 Save Done!


...


8/11 Save Done!



9/11 Save Done!



10/11 Save Done!



11/11 Save Done!


0

In [1]:
# 라이브러리 로드
import pickle as pkl
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook



link = 'D:/real/'
rg = 1000

data_range = '{}adj/data_{}.pkl'.format(link,rg)
with open(data_range, 'rb')as f:
    data = pkl.load(f)
data.sort_index(inplace=True)

col_name = []
for i in data.columns:
    col_name.append('{}_A'.format(i))
col_name.append('N')
A2 = pd.DataFrame(columns=col_name)

for i in range(0,10+1) :  # 0부터 10까지
    with open('{}adj/ddmg_adj_{}_{}.pkl'.format(link, rg, i), 'rb')as f:
        A = pkl.load(f).T
    in_name = A.index
    A = np.array(A)
    cache=[]
    for j in tqdm_notebook(np.array(data.T)) :
        cache.append((A*j).sum(axis=1))
        
    A = pd.DataFrame(A).T
    for k in A.columns:
        A[k] = A[k].apply(lambda x:x if x!=0 else np.nan)
    cache.append(A.count().tolist())
    
    A2 = pd.concat([A2, pd.DataFrame(cache, index=col_name, columns=in_name).T])

    #break # 이거만 지우면 전체 돌아감
    with open('{}adj/data_XA_{}_{}.pkl'.format(link, rg, i), 'wb') as f : 
        pkl.dump(A2, f) # 중간저장용...

with open('{}adj/data_XA_{}.pkl'.format(link, rg), 'wb') as f : 
    pkl.dump(A2, f) # 최종저장


In [ ]:
link = 'D:/real/'
rg = 1000 # 500
with open('{}adj/data_{}.pkl'.format(link, rg), 'rb')as f:
    d1 = pkl.load(f).T
with open('{}adj/data_XA_{}.pkl'.format(link, rg), 'rb')as f:
    d2 = pkl.load(f).T
with open('{}adj/y.pkl'.format(link), 'rb')as f:
    y = pkl.load(f)
data = pd.concat([d1[d2.columns], d2]).T
data.iloc[:,-1] = (data.iloc[:,-1]-data.iloc[:,-1].min())/(data.iloc[:,-1].max()-data.iloc[:,-1].min())
y = y.loc[data.index,:]


with open('{}real_data_preped/data_{}.pkl'.format(link,rg), 'wb') as f : 
    pkl.dump(data, f)
with open('{}real_data_preped/y_{}.pkl'.format(link,rg), 'wb') as f : 
    pkl.dump(y, f)

In [2]:
# 라이브러리 로드
import pickle as pkl
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook



link = 'D:/real/'
rg = 500

data_range = '{}adj/data_{}.pkl'.format(link,rg)
with open(data_range, 'rb')as f:
    data = pkl.load(f)
data.sort_index(inplace=True)

col_name = []
for i in data.columns:
    col_name.append('{}_A'.format(i))
col_name.append('N')
A2 = pd.DataFrame(columns=col_name)

for i in range(0,10+1) :  # 0부터 10까지
    with open('{}adj/ddmg_adj_{}_{}.pkl'.format(link, rg, i), 'rb')as f:
        A = pkl.load(f).T
    in_name = A.index
    A = np.array(A)
    cache=[]
    for j in tqdm_notebook(np.array(data.T)) :
        cache.append((A*j).sum(axis=1))
        
    A = pd.DataFrame(A).T
    for k in A.columns:
        A[k] = A[k].apply(lambda x:x if x!=0 else np.nan)
    cache.append(A.count().tolist())
    
    A2 = pd.concat([A2, pd.DataFrame(cache, index=col_name, columns=in_name).T])

    #break # 이거만 지우면 전체 돌아감
    with open('{}adj/data_XA_{}_{}.pkl'.format(link, rg, i), 'wb') as f : 
        pkl.dump(A2, f) # 중간저장용...

with open('{}adj/data_XA_{}.pkl'.format(link, rg), 'wb') as f : 
    pkl.dump(A2, f) # 최종저장


In [3]:
link = 'D:/real/'
rg = 500 # 1000
with open('{}adj/data_{}.pkl'.format(link, rg), 'rb')as f:
    d1 = pkl.load(f).T
with open('{}adj/data_XA_{}.pkl'.format(link, rg), 'rb')as f:
    d2 = pkl.load(f).T
with open('{}adj/y.pkl'.format(link), 'rb')as f:
    y = pkl.load(f)
data = pd.concat([d1[d2.columns], d2]).T
data.iloc[:,-1] = (data.iloc[:,-1]-data.iloc[:,-1].min())/(data.iloc[:,-1].max()-data.iloc[:,-1].min())
y = y.loc[data.index,:]


with open('{}real_data_preped/data_{}.pkl'.format(link,rg), 'wb') as f : 
    pkl.dump(data, f)
with open('{}real_data_preped/y_{}.pkl'.format(link,rg), 'wb') as f : 
    pkl.dump(y, f)

In [ ]:
# 데이터 전처리 끗! 데이터 읽기는 다음과 같음
# 데이터를 한줄씩 읽을 수 있도록 reshape 해야함!!

link = 'D:/real/'
rg = 1000 # 500
with open('{}real_data_preped/data_{}.pkl'.format(link,rg), 'rb')as f:
    X = pkl.load(f)
with open('{}real_data_preped/y_{}.pkl'.format(link,rg), 'rb')as f:
    Y = pkl.load(f)


In [4]:
# 데이터 전처리 끗! 데이터 읽기는 다음과 같음
# 데이터를 한줄씩 읽을 수 있도록 reshape 해야함!!

link = 'D:/real/'
rg = 500 # 500
with open('{}real_data_preped/data_{}.pkl'.format(link,rg), 'rb')as f:
    X = pkl.load(f)
with open('{}real_data_preped/y_{}.pkl'.format(link,rg), 'rb')as f:
    Y = pkl.load(f)
